In [1]:
# GPU: 32*40 in 8.00s = 160/s
# CPU: 32*8 in 115.0s = 2/s

In [2]:
# After installing and starting Julia run the following to install the required packages:
# Pkg.init(); Pkg.update()
# for p in ("CUDAapi","CUDAdrv","MAT","Images","IJulia","Knet"); Pkg.add(p); end
# Pkg.checkout("Knet","ilkarman") # make sure we have the right Knet version
# Pkg.build("Knet")

In [5]:
using Knet; @show gpu()
include(Knet.dir("examples","resnet", "resnetlib.jl"))
using ResNetLib: resnet50init, resnet50
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
#println("Knet: ", Pkg.installed("Knet"))

gpu() = 0


┌ Info: Precompiling MAT [23992714-dd62-5051-b70f-ba57cb901cac]
└ @ Base loading.jl:1186
ERROR: LoadError: LoadError: syntax: extra token "MatlabHDF5File" after end of expression
Stacktrace:
 [1] include at ./boot.jl:317 [inlined]
 [2] include_relative(::Module, ::String) at ./loading.jl:1038
 [3] include at ./sysimg.jl:29 [inlined]
 [4] include(::String) at /kuacc/users/dyuret/.julia/packages/MAT/KiQ9U/src/MAT.jl:27
 [5] top-level scope at none:0
 [6] include at ./boot.jl:317 [inlined]
 [7] include_relative(::Module, ::String) at ./loading.jl:1038
 [8] include(::Module, ::String) at ./sysimg.jl:29
 [9] top-level scope at none:2
 [10] eval at ./boot.jl:319 [inlined]
 [11] eval(::Expr) at ./client.jl:389
 [12] top-level scope at ./none:3
in expression starting at /kuacc/users/dyuret/.julia/packages/MAT/KiQ9U/src/MAT_HDF5.jl:39
in expression starting at /kuacc/users/dyuret/.julia/packages/MAT/KiQ9U/src/MAT.jl:31


LoadError: LoadError: LoadError: Failed to precompile MAT [23992714-dd62-5051-b70f-ba57cb901cac] to /kuacc/users/dyuret/.julia/compiled/v1.0/MAT/3FHIv.ji.
in expression starting at /kuacc/users/dyuret/.julia/dev/Knet/data/imagenet.jl:1
in expression starting at /kuacc/users/dyuret/.julia/dev/Knet/examples/resnet/resnetlib.jl:72

In [9]:
run(pipeline(`cat /proc/cpuinfo`,`grep processor`,`wc -l`));

36


In [10]:
;nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla K80


In [11]:
const BATCH_SIZE = 32
const RESNET_FEATURES = 2048
const BATCHES_GPU = 40
const BATCHES_CPU = 8

8

In [12]:
# Create batches of fake data
function fakedata(batches; atype=KnetArray)
    x = rand(Float32, 224, 224, 3, BATCH_SIZE * batches)
    minibatch(x, BATCH_SIZE, xtype=atype)
end

fakedata (generic function with 1 method)

In [13]:
# Return features from classifier
function predictfn(weights, moments, data)
    out = []
    for x in data
        pred = resnet50(weights, moments, x; stage=5)
        push!(out, mat(pred))
    end
    return Array(hcat(out...))
end

predictfn (generic function with 1 method)

## 1. GPU

In [15]:
# Initialize resnet weights and fake data
gpuweights = gpumoments = nothing; Knet.gc() # clear memory from previous run
gpuweights, gpumoments = resnet50init(;stage=5, trained=true, atype=KnetArray);

LoadError: UndefVarError: resnet50init not defined

In [17]:
@info("Cold start")
gpudata1 = fakedata(BATCHES_GPU, atype=KnetArray)
@time predictfn(gpuweights, gpumoments, gpudata1);

┌ Info: Cold start
└ @ Main In[17]:1


LoadError: UndefVarError: resnet50 not defined

In [18]:
@info("Benchmarking")
gpudata = fakedata(BATCHES_GPU, atype=KnetArray)
@time predictfn(gpuweights, gpumoments, gpudata);

┌ Info: Benchmarking
└ @ Main In[18]:1


LoadError: UndefVarError: resnet50 not defined

## 2. CPU

In [19]:
# Initialize resnet weights
cpuweights, cpumoments = resnet50init(;stage=5, trained=true, atype=Array);

LoadError: UndefVarError: resnet50init not defined

In [20]:
@info("Cold start")
cpudata1 = fakedata(1, atype=Array);
@time predictfn(cpuweights, cpumoments, cpudata1);

┌ Info: Cold start
└ @ Main In[20]:1


LoadError: UndefVarError: cpuweights not defined

In [21]:
@info("Benchmarking")
cpudata = fakedata(BATCHES_CPU, atype=Array);
@time predictfn(cpuweights, cpumoments, cpudata);

┌ Info: Benchmarking
└ @ Main In[21]:1


LoadError: UndefVarError: cpuweights not defined